In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

def preProcessamento(folder):
    images = []
    labels = []
    for labelImage in os.listdir(folder):
        DirImage = os.path.join(folder, labelImage)
        if os.path.isdir(DirImage):
            for file in os.listdir(DirImage):
                imageCarregada = os.path.join(DirImage, file)
                img = cv2.imread(imageCarregada, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img = cv2.resize(img, (256, 256)) 
                    img = cv2.equalizeHist(img) 
                    img = (img / 255.0).astype(np.float32) 
                    images.append(img)
                    labels.append(labelImage)
                else:
                    print(f"Erro carregar a imagem: {imageCarregada}")
    return np.array(images), np.array(labels)
def treinandoHog(images):
    Hog = []
    for image in tqdm(images, desc="Treinando HOG", unit="imagem"):
        Valores = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
        Hog.append(Valores)
    return np.array(Hog)
def treinandoLBP(images, P=8, R=1):
    LBP = []
    for image in tqdm(images, desc="Treinando LBP", unit="imagem"):
        image = (image * 255).astype(np.uint8) 
        lbp = local_binary_pattern(image, P, R, method='uniform')
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, P + 3), range=(0, P + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-7)
        LBP.append(hist)
    return np.array(LBP)
def concatenarCaract (model, imagens):
    Hog = treinandoHog(imagens)
    LBP = treinandoLBP(imagens)   
    Hog = np.array(Hog)
    LBP = np.array(LBP)
    return np.concatenate((Hog, LBP), axis=1)
def avalicacaoTrein(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, precision, recall, f1
trainDir               = "Training"
testDir                = "Testing"
trainImage, trainLabel = preProcessamento(trainDir)
testImage, testLabel   = preProcessamento(testDir)
label_encoder          = LabelEncoder()
trainLabel             = label_encoder.fit_transform(trainLabel)
testLabel              = label_encoder.transform(testLabel)
HogTrain               = treinandoHog (trainImage)
HogTest                = treinandoHog (testImage)
LBPTrain               = treinandoLBP(trainImage)
LBPTest                = treinandoLBP(testImage)

svmHog = SVC(kernel='linear')
svmHog.fit(HogTrain, trainLabel)
svmLBP = SVC(kernel='linear')
svmLBP.fit(LBPTrain, trainLabel)
knnHog = KNeighborsClassifier(n_neighbors=3)
knnHog.fit(HogTrain, trainLabel)
knnLBP = KNeighborsClassifier(n_neighbors=3)
knnLBP.fit(LBPTrain, trainLabel)
nbHog = GaussianNB()
nbHog.fit(HogTrain, trainLabel)
nbLBP = GaussianNB()
nbLBP.fit(LBPTrain, trainLabel)
accuracy_hog, precision_hog, recall_hog, f1_hog = avalicacaoTrein(svmHog, HogTest, testLabel)
print(f"SVM HOG: Accuracy={accuracy_hog}, Precision={precision_hog}, Recall={recall_hog}, F1-Score={f1_hog}")
accuracy_hog, precision_hog, recall_hog, f1_hog = avalicacaoTrein(knnHog, HogTest, testLabel)
print(f"KNN HOG: Accuracy={accuracy_hog}, Precision={precision_hog}, Recall={recall_hog}, F1-Score={f1_hog}")
accuracy_hog, precision_hog, recall_hog, f1_hog = avalicacaoTrein(nbHog, HogTest, testLabel)
print(f"Naive Bayes HOG: Accuracy={accuracy_hog}, Precision={precision_hog}, Recall={recall_hog}, F1-Score={f1_hog}")
accuracy_lbp, precision_lbp, recall_lbp, f1_lbp = avalicacaoTrein(svmLBP, LBPTest, testLabel)
print(f"SVM LBP: Accuracy={accuracy_lbp}, Precision={precision_lbp}, Recall={recall_lbp}, F1-Score={f1_lbp}")
accuracy_lbp, precision_lbp, recall_lbp, f1_lbp = avalicacaoTrein(knnLBP, LBPTest, testLabel)
print(f"KNN LBP: Accuracy={accuracy_lbp}, Precision={precision_lbp}, Recall={recall_lbp}, F1-Score={f1_lbp}")
accuracy_lbp, precision_lbp, recall_lbp, f1_lbp = avalicacaoTrein(nbLBP, LBPTest, testLabel)
print(f"Naive Bayes LBP: Accuracy={accuracy_lbp}, Precision={precision_lbp}, Recall={recall_lbp}, F1-Score={f1_lbp}")
